In [14]:
import pandas as pd
import pickle
! pip install spotipy

In [2]:
songs = pd.read_pickle('songs.pkl')

In [3]:
songs = songs.loc[~(songs.Title.str.contains('Album', regex = False))].copy()

In [4]:
songs = songs.loc[~(songs.Artist.str.contains('Soundtrack', regex = False))].copy()

In [5]:
songs.loc[(songs.Title.str.contains('Soundtrack', regex = False))]

,Title,Artist
12,A Charlie Brown Christmas (Soundtrack),Vince Guaraldi Trio
42,Last Christmas (Soundtrack),George Michael & Wham!
78,How The Grinch Stole Christmas (Soundtrack),Boris Karloff / Thurl Ravenscroft
95,Bohemian Rhapsody (Soundtrack),Queen
117,JESUS IS KING (Soundtrack),Kanye West
132,A Star Is Born (Soundtrack),Lady Gaga & Bradley Cooper
152,K-12 (Soundtrack),Melanie Martinez
155,The Kacey Musgraves Christmas Show (Soundtrack),Kacey Musgraves
160,Purple Rain (Soundtrack),Prince And The Revolution


In [6]:
songs.Title = songs.Title.str.replace("\(Soundtrack\)", "")

In [7]:
songs.reset_index(inplace = True)

In [8]:
songs.drop(columns = ['index'], inplace = True)

In [9]:
songs['SpotifyID'] = None

In [10]:
songs.head()

,Title,Artist,SpotifyID
0,Fine Line,Harry Styles,None
1,Christmas,Michael Buble,None
2,Please Excuse Me For Being Antisocial,Roddy Ricch,None
3,Merry Christmas,Mariah Carey,None
4,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,None


In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = "2db050f400f245738228ef384306044b"
secret = "6309c1f0455b43428d2cf142e4130a64"

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [38]:
def getSongID(trackName, trackArtist, trackResults):
    for track in trackResults['tracks']['items']:
#         print(track['id'])
#         print(track['artists'][0]['name'])
#         print(track['name'])
        if (track['name'].lower() == trackName.lower()) & (track['artists'][0]['name'].lower() == trackArtist.lower()):
            return track['id']
    return None

def getAlbumID(albumName, albumArtist, searchResult):
    for album in searchResult['albums']['items']:
        if (album['name'].lower() == albumName.lower()) & (album['artists'][0]['name'].lower() == albumArtist.lower()):
            return album['id']
    return None

def getAlbumSongs(albumArtist, albumTracksResults):
    albumSongDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])
    for songs in albumTracksResults['items']:
        newRow = pd.Series(data={"Title": songs['name'],"Artist": albumArtist, "SpotifyID": songs['id']})
        albumSongDF = albumSongDF.append(newRow, ignore_index=True)
    return albumSongDF

In [49]:
##create two df
songsToDeleteDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])
albumSongToAddDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])

#iterate through all song rows and find SpotifyID
for index, row in songs.iterrows(): 
    #get songID from Spotify Search API
    trackResults = sp.search(q=row['Title'], type='track', market= 'US', limit=10,offset=0)
    songID = getSongID(row['Title'], row['Artist'], trackResults)
    #If songID is not found, we assume it is an album and search for its albumID
    if  songID == None:
        #Delete the row from songs DF that does not have spotifyID since they are an album
        songsToDeleteDF = songsToDeleteDF.append(row, ignore_index=True)
        searchResult = sp.search(q=row['Title'], type='album', market= 'US', limit=10,offset=0)
        albumID = getAlbumID(row['Title'], row['Artist'], searchResult)
        #If we find an albumID, we get all the songs from that album and add it to the DF
        if albumID != None:
            albumTracksResults = sp.album_tracks(albumID)
            albumSongToAddDF = pd.concat([albumSongToAddDF,getAlbumSongs(row['Artist'],albumTracksResults)], ignore_index=True)
    else:
        #Add the spotifyID to an existing song
        songs.SpotifyID.iloc[index] = songID
        
        
songs = pd.concat([songs,albumSongToAddDF],ignore_index=True) #add the album songs into the song DF
songs = songs[~songs.Title.isin(songsToDeleteDF.Title)].copy() #remove album or song without spotifyID from songs DF
songs.head()

,Title,Artist,SpotifyID
0,Fine Line,Harry Styles,6VzcQuzTNTMFnJ6rBSaLH9
5,Hollywood's Bleeding,Post Malone,7sWRlDoTDX8geTR8zzr2vt
9,Lover,Taylor Swift,1dGr1c8CrMLDpV6mPbImSI
14,Rudolph The Red-Nosed Reindeer,Burl Ives,47otoIkhx3fkdivEXL5OB6
16,What You See Is What You Get,Luke Combs,273TiTHLlHSRe5zrzs7wvD


In [62]:
#songs.reset_index(inplace = True)
#songs.drop(columns = ['index'], inplace = True)
#songs.head()
#songs.to_pickle('songsWithSpotifyID.pkl')
#songs = pd.read_pickle('songsWithSpotifyID.pkl')

In [ ]:


#iterate through songs df
#for each song in song df we make a call to spotify
#loop through the spotify results to find the id BY checking the song name & artist name
#if there is a match for both, then we append the song ID to the songs df as spotify id
#if there is not song id, then we search for an album that matches the 'trackName' variable
#we get the album id and we use "getAlbumTracks" func using the album id
#put AlbumTrackResults into a temp data frame
#NOTE: we are assuming Billboard's popularity ranking, meaning all songs in the album are popular!
####
#after we iterate through all songs, we should remove songs that do not have songs id (use the pandas loc function)

In [ ]:
#TODO
#if there is not song id, then we search for an album that matches the 'trackName' variable
#we get the album id and we use "getAlbumTracks" func using the album id
#put AlbumTrackResults into a temp data frame

#build two functions, getalbumid & getalbumtracks